In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import feature_spaces
import pandas as pd
from sklearn.preprocessing import StandardScaler
from typing import List
from matplotlib import pyplot as plt
from os.path import join
import numpy as np
import encoding_utils, feature_spaces
import pickle as pkl
from collections import defaultdict
from datasets import Dataset, DatasetDict
from feature_spaces import *
NUM_VOXELS = 250

In [2]:
# Look at narrative stories
train_stories, test_stories, allstories = encoding_utils.get_allstories([1, 2, 3, 4, 5])
wordseqs = feature_spaces.get_story_wordseqs(test_stories)

In [3]:
def get_words_for_story(wordseq, max_running_words=30) -> List[str]:
    running_words = []
    
    words = np.array(wordseq.data)
    TRIM = 5
    tr_times = wordseq.tr_times[5+TRIM: -TRIM]
    for i, tr_time in enumerate(tr_times):
        valid_times = wordseq.data_times <= tr_time
        # print(valid_times)
        running_words.append(' '.join(words[valid_times][-max_running_words:]))
    #     print(tr_time, running_words)
    return running_words

texts_list_test = []
for story_name in tqdm(test_stories):
    wordseq = wordseqs[story_name]
    texts_list_test.append(get_words_for_story(wordseq))
texts_test = sum(texts_list_test, [])

100%|██████████| 1/1 [00:00<00:00, 203.45it/s]


In [4]:
subj = 'UTS03'

# select top_idxs
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/encoding/bert-10__ndel=4/UTS03'
corrs_val = np.load(join(save_dir, 'corrs.npz'))['arr_0']
top_idxs = np.argsort(corrs_val)[::-1][:NUM_VOXELS]
print(corrs_val[top_idxs][:5])

[0.76296981 0.74258237 0.72107898 0.71616266 0.71508206]


In [5]:
# load responses (n_time_points x n_voxels)
resp_test = encoding_utils.get_response(test_stories, subj)

# select top voxels
resp_test_voxel = resp_test[:, top_idxs]